In [ ]:
!pip install -q streamlit
!pip install -q streamlit-lottie
import streamlit as st
import requests
from streamlit_lottie import st_lottie

     |████████████████████████████████| 10.3 MB 5.4 MB/s 
     |████████████████████████████████| 78 kB 2.2 MB/s 
     |████████████████████████████████| 164 kB 47.2 MB/s 
     |████████████████████████████████| 237 kB 28.2 MB/s 
     |████████████████████████████████| 4.7 MB 32.3 MB/s 
     |████████████████████████████████| 182 kB 52.0 MB/s 
     |████████████████████████████████| 62 kB 759 kB/s 
     |████████████████████████████████| 51 kB 3.4 MB/s 
     |████████████████████████████████| 769 kB 5.1 MB/s 


In [ ]:
# ---- Paragraph ----
with st.container():
    st.write("---")
    left_column, right_column = st.columns(2)
    with left_column:
        st.header("What I do")
        st.write("##")
        st.write(
            """
            On my YouTube channel I am creating tutorials for people who:
            - are looking for a way to leverage the power of Python in their day-to-day work.
            - are struggling with repetitive tasks in Excel and are looking for a way to use Python and VBA.
            - want to learn Data Analysis & Data Science to perform meaningful and impactful analyses.
            - are working with Excel and found themselves thinking - "there has to be a better way."
            If this sounds interesting to you, consider subscribing and turning on the notifications, so you don’t miss any content.
            """
        )
        st.write("[YouTube Channel >](https://youtube.com/c/CodingIsFun)")
    with right_column:
        st_lottie(lottie_coding, height=300, key="coding")

NameError: ignored

# App Py

In [58]:
%%writefile app.py
import streamlit as st
import requests

import pickle
import pandas as pd
from streamlit_lottie import st_lottie

restros_dict = pickle.load(open('restros.pkl','rb'))
restros = pd.DataFrame(restros_dict)
df_percent = pd.read_csv('z3_new.csv')
#df_percent = pd.DataFrame(restros_dict)
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)
cosine_similarities = pickle.load(open('similarity.pkl','rb'))
show_df = False

# --------- Functions ---------------
def load_lottieurl(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    #df_new = df_new.style.set_properties(**{'text-align': 'left'})
    
    return df_new



# --------- Assets ---------------------


# ---------Title and Header --------------
st.set_page_config(page_title="RPS One", layout = "wide")

with st.container():
    st.title("CHEF WISE: A Restaurant Prediction System")
    st.header("Create a customized list of restaurants curated for you!")
    #st.write("Providing the best in business Restaurant recommandations in Bangalore")


# ---- Paragraph ----
with st.container():
  left_column, right_column = st.columns(2)
  with left_column:
    selected_movie_name = st.selectbox(
      "Type or select a diner from the dropdown",
      restros['name'].values
    )
  if st.button('Show Recommendation'):
    answer = recommend(selected_movie_name)
    show_df = True
    st.dataframe(answer)

st.header("How it works")
st.write("We find restros similar to your input using ML technology which you can sort by mean rating or price per person")
    


Overwriting app.py


# Restaurant Predictor


In [47]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_percent = pd.read_csv('z3_new.csv')
print(type(df_percent))

<class 'pandas.core.frame.DataFrame'>


In [53]:
df_percent.shape

(4496, 10)

In [ ]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'].values.astype('U'))

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
import pickle
pickle.dump(df_percent.to_dict(),open('restros.pkl','wb'))
pickle.dump(cosine_similarities,open('similarity.pkl','wb'))

In [49]:
import pickle
restros_dict1 = pickle.load(open('restros.pkl','rb'))
restros1 = pd.DataFrame(restros_dict1)

In [52]:
df_percent

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,TRUE,TRUE,4.1,Banashankari,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99
1,Spice Elephant,TRUE,FALSE,4.1,Banashankari,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,Banashankari,3.97
2,San Churro Cafe,TRUE,FALSE,3.8,Banashankari,"Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,Banashankari,3.58
3,Addhuri Udupi Bhojana,FALSE,FALSE,3.7,Banashankari,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,Banashankari,3.45
4,Grand Village,FALSE,FALSE,3.8,Basavanagudi,"North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,3.58
...,...,...,...,...,...,...,...,...,...,...
4491,NaN,FALSE,TRUE,3.9,Shanti Nagar,"Cafe, Italian",750.0,rated 30 ratedn cafe situated right middle one...,Brigade Road,3.71
4492,NaN,TRUE,FALSE,3.9,Residency Road,"North Indian, Chinese, Biryani, Kebab, Juices",800.0,rated 50 ratedn love kababs ask wings legs ver...,Brigade Road,3.50
4493,NaN,TRUE,FALSE,4.2,Richmond Road,Cafe,700.0,rated 40 ratedn could even miss gem middle cit...,Brigade Road,4.10
4494,NaN,TRUE,FALSE,3.6,Infantry Road,"Cafe, Burger, Italian, Fast Food",650.0,rated 10 ratedn friend mine lunch absolutely a...,Brigade Road,3.37


In [50]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    #df_new = df_new.style.set_properties(**{'text-align': 'left'})
    
    return df_new


In [51]:
one = recommend('Spice Elephant')
display(one)

TOP 10 RESTAURANTS LIKE Spice Elephant WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
693,"North Indian, Chinese, Mughlai",3.71,700.0
1051,Chinese,3.58,600.0
1992,"Fast Food, North Indian, South Indian",3.45,200.0
3493,"North Indian, Biryani, Chinese",3.45,550.0
1362,"Chinese, North Indian",3.45,550.0
4042,"Fast Food, South Indian, Biryani",3.19,300.0
2893,"North Indian, Biryani",2.55,800.0
3503,"North Indian, Biryani, Chinese",NaN,600.0
3539,"North Indian, Biryani",NaN,800.0
3818,"North Indian, Chinese",NaN,500.0


In [ ]:
display(movies1)

In [ ]:
import streamlit as st
two = recommend('Jalsa')
display(two)
st.dataframe(two,200,100)

TOP 10 RESTAURANTS LIKE Jalsa WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
2254,"Cafe, Beverages",4.61,400.0
2964,"Pizza, Cafe, Italian",4.37,600.0
2279,"Cafe, Continental, Sandwich, Beverages",4.15,800.0
3760,"Cafe, American, Italian, Pizza, Burger",3.98,700.0
1934,"North Indian, Mughlai, South Indian, Chinese",3.88,750.0
68,"North Indian, Chinese",3.87,850.0
2748,Cafe,3.58,400.0
1992,"Fast Food, North Indian, South Indian",3.45,200.0
4042,"Fast Food, South Indian, Biryani",3.19,300.0
4188,North Indian,2.94,200.0


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

# Run Site


In [59]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] | fetchMetadata: sill resolveWithNewModule ansi-styles@4.3


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.74.5.9:8501

npx: installed 22 in 3.082s
your url is: https://evil-carrots-camp-34-74-5-9.loca.lt
  Stopping...
^C


# Creating Similairty Matrix